In [ ]:
# !pip install ortools

In [2]:
# %% Method 2

import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

# --- 1. Load DataFrames ---
df_orders = pd.read_csv('orders_day1.csv')
df_distance = pd.read_excel("distance_matrix_1.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_1.xlsx', index_col=0)

# --- PARAMETERS BASED ON MIP FORMULATION ---
# Alpha (α): Coefficient to penalize the number of vehicles. Must be large to prioritize
# vehicle count minimization over total travel time minimization.
ALPHA_VEHICLE_PENALTY = 200

# Physical Fleet Specifications (for sanity checks and fleet sizing)
PHYSICAL_FLEET_SPECS = {
    'weight': 2000,  # Max weight capacity of each truck
    'volume': 1000   # Max volume capacity of each truck
}

# Buffer time to ensure depot is open when vehicles return
BUFFER_RETURN_TIME = 200

# Extra vehicles to add beyond theoretical minimum
ROUTING_BUFFER = 5

# ----------------------------------------------------------------------
# --- 2. Data Structure Preparation for OR-Tools ---
# ----------------------------------------------------------------------

import math

def create_data_model_smart(orders_df, distance_df, time_df, physical_fleet_specs={}):
    """
    Initializes VRP data dynamically based on input statistics to ensure feasibility.

    physical_fleet_specs: dict containing 'weight_limit', 'volume_limit' of your REAL trucks.
    """
    data = {}

    # --- 1. Data Cleaning (Keep existing logic) ---
    distance_df.dropna(how='all', axis=0, inplace=True)
    distance_df.dropna(how='all', axis=1, inplace=True)
    time_df.dropna(how='all', axis=0, inplace=True)
    time_df.dropna(how='all', axis=1, inplace=True)

    data['distance_matrix'] = distance_df.values.astype(int).tolist()
    data['time_matrix'] = time_df.values.astype(int).tolist()

    # --- 2. Smart Time Window Parsing ---
    # Parse actual windows first to find the "Horizon" (Latest possible deadline)
    parsed_windows = []
    max_deadline_in_data = 0

    # Depot is always (0,0) or (0, Open_Duration)
    # Let's assume Depot is open as long as the latest customer needs.
    parsed_windows.append((0, 0))

    for tw_str in orders_df['TIME WINDOW']:
        # Parse the string "(900, 1200)" -> 900, 1200
        clean_str = tw_str.strip('()')
        if ',' in clean_str:
            parts = clean_str.split(',')
            start = int(parts[0])
            end = int(parts[1])
            parsed_windows.append((start, end))
            # Track the latest time anyone needs service
            if end > max_deadline_in_data:
                max_deadline_in_data = end
        else:
            # Fallback for bad data
            parsed_windows.append((0, 10000)) # Default fallback

    # Update Depot's window to extend to the latest deadline (plus return trip buffer)
    # This prevents the "Depot closed before driver returns" error.
    # approx time to drive back to depot after last delivery
    horizon = max_deadline_in_data + BUFFER_RETURN_TIME
    parsed_windows[0] = (0, horizon)

    data['time_windows'] = parsed_windows
    data['vehicle_max_travel_time'] = horizon # Set Horizon dynamically

    # --- 3. Demand & Capacity Sanity Check ---
    data['weights'] = [0] + orders_df['WEIGHT'].round().astype(int).tolist()

    # Notice: Multiplier is consistent (was 100 in your snippet, 1000 in previous. Check your data!)
    data['volumes'] = [0] + (orders_df['VOLUME'] * 100).round().astype(int).tolist()

    # CONSTANTS (Physical limits of your trucks)
    TRUCK_W_CAP = physical_fleet_specs.get('weight', 2000)
    TRUCK_V_CAP = physical_fleet_specs.get('volume', 1000)

    # Sanity Check: Does the biggest order fit in a truck?
    max_order_w = max(data['weights'])
    if max_order_w > TRUCK_W_CAP:
        raise ValueError(f"CRITICAL ERROR: Order exists with weight {max_order_w}, but truck limit is {TRUCK_W_CAP}.")

    # --- 4. Smart Fleet Sizing (The Lower Bound Calculation) ---
    total_weight = sum(data['weights'])
    total_volume = sum(data['volumes'])

    # Minimum trucks needed purely for capacity (Bin Packing Lower Bound)
    min_trucks_weight = math.ceil(total_weight / TRUCK_W_CAP)
    min_trucks_volume = math.ceil(total_volume / TRUCK_V_CAP)

    theoretical_min_vehicles = max(min_trucks_weight, min_trucks_volume)

    # Add a "Routing Buffer" (e.g., 20% or +2 trucks)
    # Vehicles can rarely be 100% full because they run out of Time or Distance first.
    recommended_fleet_size = int(theoretical_min_vehicles * 1.2) + ROUTING_BUFFER

    print(f"--- Initialization Report ---")
    print(f"Total Weight: {total_weight} | Max Truck W: {TRUCK_W_CAP} -> Min Trucks: {min_trucks_weight}")
    print(f"Total Volume: {total_volume} | Max Truck V: {TRUCK_V_CAP} -> Min Trucks: {min_trucks_volume}")
    print(f"Latest Deadline found: {max_deadline_in_data}")
    print(f"Setting Fleet Size to: {recommended_fleet_size} (Theoretical Min: {theoretical_min_vehicles})")

    data['num_vehicles'] = recommended_fleet_size
    data['vehicle_capacities_weight'] = [TRUCK_W_CAP] * data['num_vehicles']
    data['vehicle_capacities_volume'] = [TRUCK_V_CAP] * data['num_vehicles']

    # --- 5. Other Data ---
    data['service_times'] = [0] + orders_df['SERVICE_TIME'].astype(int).tolist()
    data['depot'] = 0
    data['penalty'] = 100000 # Keep high

    return data

# ----------------------------------------------------------------------
# --- 3. Initialize Solver Model and Constraints ---
# ----------------------------------------------------------------------

def print_solution(data, manager, routing, solution):
   """Prints the solution found by the solver."""
   total_distance = 0
   total_time_cost = 0 # Cost is now based on time, not distance
   total_time = 0
   time_dimension = routing.GetDimensionOrDie('Time')

   # Calculate the total travel time for the objective reporting
   for vehicle_id in range(data['num_vehicles']):
       index = routing.Start(vehicle_id)
       if routing.IsEnd(solution.Value(routing.NextVar(index))):
           continue

       while not routing.IsEnd(index):
           previous_index = index
           index = solution.Value(routing.NextVar(index))

           # The actual total travel time component of the objective
           from_node = manager.IndexToNode(previous_index)
           to_node = manager.IndexToNode(index)
           # The cost being minimized is based on the time matrix
           total_time_cost += data['time_matrix'][from_node][to_node]

   # Calculate objective based on MIP formulation: alpha * Yk + sum(tij * xijk)
   num_used_vehicles = len([v for v in range(data['num_vehicles']) if not routing.IsEnd(solution.Value(routing.NextVar(routing.Start(v))))])
   vehicle_penalty_component = num_used_vehicles * ALPHA_VEHICLE_PENALTY

   # NOTE: The OR-Tools objective value here may include dropped node penalties,
   # but the custom calculation below reflects the MIP goal:
   mip_objective = vehicle_penalty_component + total_time_cost

   print(f'OR-Tools Objective (Time Cost + Penalties): {solution.ObjectiveValue()}')
   print(f'MIP Objective (Alpha*Vehicles + Total Time): {mip_objective} ({num_used_vehicles} vehicles * {ALPHA_VEHICLE_PENALTY} + {total_time_cost})')

   # Print routes (unchanged)
   for vehicle_id in range(data['num_vehicles']):
       index = routing.Start(vehicle_id)
       if routing.IsEnd(solution.Value(routing.NextVar(index))):
           continue

       plan_output = f'Route for vehicle {vehicle_id} (Depot: 0):'
       route_distance = 0

       while not routing.IsEnd(index):
           time_var = time_dimension.CumulVar(index)
           node_index = manager.IndexToNode(index)

           previous_index = index
           index = solution.Value(routing.NextVar(index))

           route_distance += routing.GetArcCostForVehicle(
               previous_index, index, vehicle_id
           )

           plan_output += (
               f' {node_index} -> Time({solution.Min(time_var)})'
           )

       time_var = time_dimension.CumulVar(index)
       plan_output += (
           f' {manager.IndexToNode(index)} -> Time({solution.Min(time_var)})'
       )
       plan_output += f'\n  Route Distance: {route_distance}'
       plan_output += f'\n  Route End Time: {solution.Min(time_var)}'

       print(plan_output)
       total_distance += route_distance
       total_time = max(total_time, solution.Min(time_var))

   print(f'\nTotal distance of all used routes: {total_distance}')
   print(f'Max route end time: {total_time}')
   print(f'Total travel time cost: {total_time_cost} (The minimized sum component)')

   # --- Report Dropped Nodes ---
   dropped_nodes = []
#    for node in range(1, len(data['distance_matrix'])):
#        if solution.Value(routing.NextVar(manager.NodeToIndex(node))) == manager.NodeToIndex(node):
#            dropped_nodes.append(node)

   if dropped_nodes:
       print(f'\n!!! WARNING: {len(dropped_nodes)} Nodes Were DROPPED (Unserved) !!!')
       print(f'Dropped Nodes (NODE_ID): {dropped_nodes}')
   else:
       print('\nSUCCESS: All nodes were served.')


def solve_vrp():
   """Entry point for the VRP solver with MIP objective."""
   data = create_data_model_smart(df_orders, df_distance, df_time)

   manager = pywrapcp.RoutingIndexManager(
       len(data['distance_matrix']), data['num_vehicles'], data['depot']
   )

   routing = pywrapcp.RoutingModel(manager)

   # --- A. Define Cost (Time - as per MIP Objective) ---
   def time_cost_callback(from_index, to_index):
       from_node = manager.IndexToNode(from_index)
       to_node = manager.IndexToNode(to_index)
       # The cost minimized is the travel time (t_ij)
       return data['time_matrix'][from_node][to_node]

   transit_callback_index = routing.RegisterTransitCallback(time_cost_callback)
   routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

   # A.1. Set Objective to Minimize Time Cost (t_ij) AND Penalize Vehicles (alpha * y_k)

   # Fixed cost (alpha * y_k): Apply a large penalty for using each vehicle
   for vehicle_id in range(data['num_vehicles']):
       routing.SetFixedCostOfVehicle(ALPHA_VEHICLE_PENALTY, vehicle_id)

   # A.2. Add Dropped Node Penalty
   for node in range(1, len(data['distance_matrix'])):
       routing.AddDisjunction([manager.NodeToIndex(node)], data['penalty'])

   # --- B. Add Capacity Constraints (Weight and Volume) ---

   def add_capacity_dimension(capacity_name, capacities, demands):
       def demand_callback(index):
           node = manager.IndexToNode(index)
           return demands[node]

       demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

       # Capacity bounds are imposed at every node [cite: 87]
       routing.AddDimensionWithVehicleCapacity(
           demand_callback_index,
           0, # Slack
           capacities,
           True, # This dimension is cumulative
           capacity_name
       )

   add_capacity_dimension('WeightCapacity', data['vehicle_capacities_weight'], data['weights'])
   add_capacity_dimension('VolumeCapacity', data['vehicle_capacities_volume'], data['volumes'])


   # --- C. Add Time Window Constraints ---

   def time_callback(from_index, to_index):
       from_node = manager.IndexToNode(from_index)
       to_node = manager.IndexToNode(to_index)
       travel_time = data['time_matrix'][from_node][to_node]
       service_time = data['service_times'][from_node]
       # Time propagation: T_jk >= T_ik + s_i + t_ij [cite: 99]
       return travel_time + service_time

   time_callback_index = routing.RegisterTransitCallback(time_callback)

   # Time dimension: tracks T_ik (time vehicle k starts service at node i) [cite: 45]
   routing.AddDimension(
       time_callback_index,
       0,                             # slack_max (0 is fine here)
       data['vehicle_max_travel_time'], # Planning horizon H [cite: 28]
       False,
       'Time'
   )
   time_dimension = routing.GetDimensionOrDie('Time')

   # Apply Time Windows [cite: 94]
   for node in range(len(data['time_windows'])):
       index = manager.NodeToIndex(node)
       start, end = data['time_windows'][node]
       time_dimension.CumulVar(index).SetRange(start, end)

   # Apply Route Duration Constraint (limited by H) [cite: 104]
   for i in range(data['num_vehicles']):
       time_dimension.SetSpanUpperBoundForVehicle(
           data['vehicle_max_travel_time'],
           i
       )

   # --- D. Set Search Parameters and Solve ---
   search_parameters = pywrapcp.DefaultRoutingSearchParameters()
   search_parameters.first_solution_strategy = (
       routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
   )
   search_parameters.local_search_metaheuristic = (
       routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
   )
   search_parameters.time_limit.seconds = 120

   # Solve the problem
   solution = routing.SolveWithParameters(search_parameters)

   # --- E. Print Solution ---
   if solution:
       print_solution(data, manager, routing, solution)
   else:
       print('No solution found!')


# ----------------------------------------------------------------------
# --- EXECUTION ---
# ----------------------------------------------------------------------

# print("Running Vehicle Routing Problem Solver (MIP Objective: Minimize Vehicles, then Time)...")
# print("Day 1:")
# solve_vrp()

In [3]:
def check_day(df_orders, df_distance, df_time, day):
    print(f"Shapes Day {day}: orders={df_orders.shape}, dist={df_distance.shape}, time={df_time.shape}")

    assert df_distance.shape[0] == df_distance.shape[1], "Distance matrix not square"
    assert df_time.shape[0] == df_time.shape[1], "Time matrix not square"
    assert df_distance.shape == df_time.shape, "Distance and time shapes differ"

    n_orders = len(df_orders)
    n_nodes = df_distance.shape[0]
    assert n_nodes == n_orders + 1, f"Expected matrix size {n_orders+1}x{n_orders+1} (depot+orders), got {n_nodes}x{n_nodes}"

print("Day 1:")
check_day(df_orders, df_distance, df_time, 1)
solve_vrp()

Day 1:
Shapes Day 1: orders=(78, 7), dist=(79, 79), time=(79, 79)
--- Initialization Report ---
Total Weight: 1877 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 898 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1091
MIP Objective (Alpha*Vehicles + Total Time): 1091 (3 vehicles * 200 + 491)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 2 -> Time(18) 4 -> Time(32) 26 -> Time(41) 28 -> Time(62) 24 -> Time(77) 25 -> Time(89) 53 -> Time(96) 52 -> Time(110) 23 -> Time(123) 19 -> Time(131) 55 -> Time(138) 20 -> Time(156) 22 -> Time(172) 50 -> Time(180) 21 -> Time(187) 49 -> Time(196) 48 -> Time(205) 47 -> Time(211) 45 -> Time(220) 44 -> Time(230) 46 -> Time(240) 42 -> Time(248) 43 -> Time(257) 41 -> Time(267) 60 -> Time(283) 75 -> Time(291) 62 -> Time(300) 61 -> Time(312) 74 -> Time(321) 73 -> Time(331) 70 -> Time(345) 71 -> Time(355) 0 -> Time(387)
  Route Distance: 359
  Route End

In [4]:
print("\n\nDay 2:")
df_orders = pd.read_csv('orders_day2.csv')
df_distance = pd.read_excel("distance_matrix_2.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_2.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 2)
solve_vrp()



Day 2:
Shapes Day 2: orders=(63, 7), dist=(64, 64), time=(64, 64)
--- Initialization Report ---
Total Weight: 1394 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 519 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1130
MIP Objective (Alpha*Vehicles + Total Time): 1130 (3 vehicles * 200 + 530)
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 58 -> Time(17) 44 -> Time(38) 48 -> Time(48) 45 -> Time(55) 49 -> Time(77) 47 -> Time(93) 57 -> Time(102) 61 -> Time(126) 62 -> Time(146) 60 -> Time(152) 63 -> Time(159) 43 -> Time(178) 19 -> Time(206) 18 -> Time(211) 0 -> Time(241)
  Route Distance: 341
  Route End Time: 241
Route for vehicle 2 (Depot: 0): 0 -> Time(0) 34 -> Time(21) 8 -> Time(55) 40 -> Time(87) 51 -> Time(98) 50 -> Time(108) 4 -> Time(127) 6 -> Time(136) 7 -> Time(154) 9 -> Time(168) 15 -> Time(176) 10 -> Time(181) 55 -> Time(192) 52 -> Time(199) 53 -> Time(204) 5 -> Time(21

In [5]:
print("\n\nDay 3:")
df_orders = pd.read_csv('orders_day3.csv')
df_distance = pd.read_excel("distance_matrix_3.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_3.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 3)
solve_vrp()



Day 3:
Shapes Day 3: orders=(67, 7), dist=(68, 68), time=(68, 68)
--- Initialization Report ---
Total Weight: 682 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 401 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1088
MIP Objective (Alpha*Vehicles + Total Time): 1088 (3 vehicles * 200 + 488)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 50 -> Time(16) 42 -> Time(25) 46 -> Time(34) 17 -> Time(49) 20 -> Time(63) 23 -> Time(77) 22 -> Time(86) 21 -> Time(100) 2 -> Time(126) 6 -> Time(135) 3 -> Time(152) 5 -> Time(157) 7 -> Time(164) 39 -> Time(192) 40 -> Time(198) 44 -> Time(205) 41 -> Time(214) 0 -> Time(232)
  Route Distance: 336
  Route End Time: 232
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 31 -> Time(18) 26 -> Time(25) 25 -> Time(42) 24 -> Time(55) 8 -> Time(82) 9 -> Time(92) 4 -> Time(107) 15 -> Time(117) 13 -> Time(127) 16 -> Time(133) 14 -> Time(138) 11 -> Time(150) 12 

In [6]:
print("\n\nDay 4:")
df_orders = pd.read_csv('orders_day4.csv')
df_distance = pd.read_excel("distance_matrix_4.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_4.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 4)
solve_vrp()



Day 4:
Shapes Day 4: orders=(69, 7), dist=(70, 70), time=(70, 70)
--- Initialization Report ---
Total Weight: 1690 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 469 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1382
MIP Objective (Alpha*Vehicles + Total Time): 1382 (4 vehicles * 200 + 582)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 30 -> Time(31) 31 -> Time(47) 57 -> Time(63) 53 -> Time(72) 54 -> Time(87) 55 -> Time(95) 46 -> Time(108) 41 -> Time(116) 43 -> Time(129) 44 -> Time(137) 40 -> Time(144) 45 -> Time(152) 42 -> Time(168) 49 -> Time(181) 48 -> Time(186) 50 -> Time(191) 51 -> Time(202) 47 -> Time(211) 52 -> Time(223) 69 -> Time(238) 63 -> Time(264) 0 -> Time(291)
  Route Distance: 359
  Route End Time: 291
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 8 -> Time(21) 7 -> Time(36) 28 -> Time(62) 29 -> Time(69) 26 -> Time(75) 32 -> Time(92) 27 -> Time(108) 56 -> Time(

In [11]:
print("\n\nDay 5:")
df_orders = pd.read_csv('orders_day5.csv')
df_orders.columns = df_orders.columns.str.strip() 

if 'TIME WINDOW' not in df_orders.columns and 'TIME_WINDOW' in df_orders.columns:
    df_orders = df_orders.rename(columns={'TIME_WINDOW': 'TIME WINDOW'})
df_distance = pd.read_excel("distance_matrix_5.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_5.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 5)
solve_vrp()



Day 5:
Shapes Day 5: orders=(75, 7), dist=(76, 76), time=(76, 76)
--- Initialization Report ---
Total Weight: 1271 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 446 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1390
MIP Objective (Alpha*Vehicles + Total Time): 1390 (4 vehicles * 200 + 590)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 47 -> Time(25) 45 -> Time(31) 46 -> Time(36) 48 -> Time(45) 49 -> Time(53) 51 -> Time(60) 50 -> Time(69) 44 -> Time(81) 42 -> Time(96) 43 -> Time(115) 40 -> Time(139) 33 -> Time(156) 38 -> Time(175) 37 -> Time(197) 56 -> Time(202) 41 -> Time(216) 16 -> Time(235) 0 -> Time(264)
  Route Distance: 348
  Route End Time: 264
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 9 -> Time(19) 60 -> Time(36) 57 -> Time(46) 58 -> Time(55) 62 -> Time(63) 59 -> Time(72) 63 -> Time(82) 65 -> Time(98) 61 -> Time(106) 36 -> Time(117) 39 -> Time(138) 30 -> Time(169)

In [13]:
print("\n\nDay 6:")
df_orders = pd.read_csv('orders_day6.csv')
df_orders.columns = df_orders.columns.str.strip() 

if 'TIME WINDOW' not in df_orders.columns and 'TIME_WINDOW' in df_orders.columns:
    df_orders = df_orders.rename(columns={'TIME_WINDOW': 'TIME WINDOW'})
df_distance = pd.read_excel("distance_matrix_6.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_6.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 6)
solve_vrp()



Day 6:
Shapes Day 6: orders=(77, 7), dist=(78, 78), time=(78, 78)
--- Initialization Report ---
Total Weight: 1302 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 455 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1156
MIP Objective (Alpha*Vehicles + Total Time): 1156 (3 vehicles * 200 + 556)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 75 -> Time(18) 73 -> Time(24) 67 -> Time(45) 37 -> Time(65) 39 -> Time(82) 36 -> Time(96) 35 -> Time(112) 46 -> Time(129) 43 -> Time(146) 40 -> Time(155) 48 -> Time(172) 44 -> Time(181) 42 -> Time(190) 47 -> Time(205) 45 -> Time(212) 49 -> Time(219) 41 -> Time(233) 64 -> Time(247) 60 -> Time(255) 70 -> Time(263) 61 -> Time(270) 63 -> Time(276) 62 -> Time(283) 59 -> Time(296) 0 -> Time(321)
  Route Distance: 377
  Route End Time: 321
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 52 -> Time(19) 58 -> Time(31) 57 -> Time(37) 77 -> Time(58) 38 -> T

In [14]:
print("\n\nDay 7:")
df_orders = pd.read_csv('orders_day7.csv')
df_orders.columns = df_orders.columns.str.strip() 

if 'TIME WINDOW' not in df_orders.columns and 'TIME_WINDOW' in df_orders.columns:
    df_orders = df_orders.rename(columns={'TIME_WINDOW': 'TIME WINDOW'})
    
df_distance = pd.read_excel("distance_matrix_7.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_7.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 7)
solve_vrp()



Day 7:
Shapes Day 7: orders=(66, 7), dist=(67, 67), time=(67, 67)
--- Initialization Report ---
Total Weight: 1030 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 359 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1151
MIP Objective (Alpha*Vehicles + Total Time): 1151 (3 vehicles * 200 + 551)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 3 -> Time(14) 8 -> Time(19) 2 -> Time(31) 7 -> Time(46) 1 -> Time(61) 9 -> Time(72) 55 -> Time(99) 53 -> Time(116) 58 -> Time(136) 60 -> Time(144) 50 -> Time(154) 62 -> Time(166) 59 -> Time(187) 57 -> Time(200) 61 -> Time(208) 63 -> Time(223) 64 -> Time(242) 49 -> Time(253) 48 -> Time(259) 46 -> Time(266) 66 -> Time(276) 0 -> Time(298)
  Route Distance: 370
  Route End Time: 298
Route for vehicle 2 (Depot: 0): 0 -> Time(0) 6 -> Time(24) 41 -> Time(39) 29 -> Time(55) 31 -> Time(78) 51 -> Time(88) 28 -> Time(96) 26 -> Time(102) 27 -> Time(107) 5

In [15]:
print("\n\nDay 8:")
df_orders = pd.read_csv('orders_day8.csv')
df_orders.columns = df_orders.columns.str.strip() 

if 'TIME WINDOW' not in df_orders.columns and 'TIME_WINDOW' in df_orders.columns:
    df_orders = df_orders.rename(columns={'TIME_WINDOW': 'TIME WINDOW'})
    
df_distance = pd.read_excel("distance_matrix_8.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_8.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 8)
solve_vrp()



Day 8:
Shapes Day 8: orders=(74, 7), dist=(75, 75), time=(75, 75)
--- Initialization Report ---
Total Weight: 1090 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 318 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1440
MIP Objective (Alpha*Vehicles + Total Time): 1440 (4 vehicles * 200 + 640)
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 40 -> Time(19) 25 -> Time(36) 50 -> Time(56) 49 -> Time(70) 48 -> Time(90) 47 -> Time(104) 45 -> Time(121) 33 -> Time(132) 32 -> Time(141) 29 -> Time(149) 41 -> Time(162) 43 -> Time(179) 42 -> Time(185) 20 -> Time(198) 19 -> Time(206) 15 -> Time(219) 14 -> Time(229) 0 -> Time(258)
  Route Distance: 366
  Route End Time: 258
Route for vehicle 2 (Depot: 0): 0 -> Time(0) 72 -> Time(23) 52 -> Time(35) 56 -> Time(47) 57 -> Time(57) 54 -> Time(62) 55 -> Time(67) 58 -> Time(77) 59 -> Time(86) 60 -> Time(95) 61 -> Time(106) 62 -> Time(112) 63 -> Time(

In [16]:
print("\n\nDay 9:")
df_orders = pd.read_csv('orders_day9.csv')
df_orders.columns = df_orders.columns.str.strip() 

if 'TIME WINDOW' not in df_orders.columns and 'TIME_WINDOW' in df_orders.columns:
    df_orders = df_orders.rename(columns={'TIME_WINDOW': 'TIME WINDOW'})
    
df_distance = pd.read_excel("distance_matrix_9.xlsx", index_col=0)
df_time = pd.read_excel('time_matrix_mostlikely_9.xlsx', index_col=0)

check_day(df_orders, df_distance, df_time, 9)
solve_vrp()



Day 9:
Shapes Day 9: orders=(84, 7), dist=(85, 85), time=(85, 85)
--- Initialization Report ---
Total Weight: 1303 | Max Truck W: 2000 -> Min Trucks: 1
Total Volume: 524 | Max Truck V: 1000 -> Min Trucks: 1
Latest Deadline found: 360
Setting Fleet Size to: 6 (Theoretical Min: 1)
OR-Tools Objective (Time Cost + Penalties): 1452
MIP Objective (Alpha*Vehicles + Total Time): 1452 (4 vehicles * 200 + 652)
Route for vehicle 0 (Depot: 0): 0 -> Time(0) 28 -> Time(30) 27 -> Time(43) 29 -> Time(51) 25 -> Time(66) 52 -> Time(81) 30 -> Time(97) 31 -> Time(110) 26 -> Time(131) 24 -> Time(145) 21 -> Time(156) 23 -> Time(170) 65 -> Time(189) 22 -> Time(196) 75 -> Time(212) 74 -> Time(224) 73 -> Time(238) 72 -> Time(245) 76 -> Time(255) 81 -> Time(269) 80 -> Time(278) 77 -> Time(288) 79 -> Time(297) 78 -> Time(306) 0 -> Time(339)
  Route Distance: 351
  Route End Time: 339
Route for vehicle 1 (Depot: 0): 0 -> Time(0) 61 -> Time(19) 19 -> Time(50) 20 -> Time(60) 82 -> Time(71) 15 -> Time(79) 17 -> Ti